## Loading the Data

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset('billingsmoore/84000-bo-en')

{'bo': 'བཅོམ་ལྡན་འདས་ཅི་དེ་བཞིན་གཤེགས་པ་དགྲ་བཅོམ་པ་ཡང་དག་པར་རྫོགས་པའི་སངས་རྒྱས་རྣམས་ཡོན་ཏན་དཔག་ཏུ་མ་མཆིས་པ་དང་ལྡན་ལགས་སམ། དེ་སྐད་ཅེས་གསོལ་པ་དང་།',
 'en': 'Blessed One, do thus-gone, worthy, perfectly complete buddhas possess limitless good qualities?”',
 'source_file': 'toh126'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [5]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('billingsmoore/getok-v0')
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-efficient-tiny", device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

## Preprocess Data

The dataset can now be tokenized for training.

In [4]:
def bo_en_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['tibetan']]
    translation_targets = [example for example in examples['english']]
    
    # Tokenize translation inputs and targets
    bo_en_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return bo_en_model_inputs

In [5]:
def en_bo_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate English to Tibetan: ' + example for example in examples['english']]
    translation_targets = [example for example in examples['tibetan']]
    
    # Tokenize translation inputs and targets
    en_bo_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return en_bo_model_inputs

In [6]:
bo_en_tokenized_dataset = dataset.map(bo_en_preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map:   0%|          | 0/12184 [00:00<?, ? examples/s]

In [7]:
en_bo_tokenized_dataset = dataset.map(en_bo_preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map:   0%|          | 0/12184 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset = {}

tokenized_dataset['train'] = concatenate_datasets([bo_en_tokenized_dataset['train'], en_bo_tokenized_dataset['train']])
tokenized_dataset['test'] = bo_en_tokenized_dataset['test']

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


True

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"bidirectional",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/51783 [00:00<?, ?it/s]

{'loss': 0.6332, 'grad_norm': 0.17786581814289093, 'learning_rate': 0.0002971032964486414, 'epoch': 0.03}
{'loss': 0.299, 'grad_norm': 0.12014766037464142, 'learning_rate': 0.00029420659289728285, 'epoch': 0.06}
{'loss': 0.2795, 'grad_norm': 0.16227394342422485, 'learning_rate': 0.0002913098893459243, 'epoch': 0.09}
{'loss': 0.2722, 'grad_norm': 0.21368838846683502, 'learning_rate': 0.00028841318579456577, 'epoch': 0.12}
{'loss': 0.2595, 'grad_norm': 0.14164778590202332, 'learning_rate': 0.0002855164822432072, 'epoch': 0.14}
{'loss': 0.2529, 'grad_norm': 0.1506086140871048, 'learning_rate': 0.00028261977869184864, 'epoch': 0.17}
{'loss': 0.2533, 'grad_norm': 0.14093157649040222, 'learning_rate': 0.0002797230751404901, 'epoch': 0.2}
{'loss': 0.2459, 'grad_norm': 0.13410639762878418, 'learning_rate': 0.0002768263715891315, 'epoch': 0.23}
{'loss': 0.2392, 'grad_norm': 0.16665083169937134, 'learning_rate': 0.00027392966803777295, 'epoch': 0.26}
{'loss': 0.2356, 'grad_norm': 0.1214125454425

/home/j/Desktop/MLotsawa/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1523 [00:00<?, ?it/s]

...

  0%|          | 0/1523 [00:00<?, ?it/s]

{'eval_loss': 0.1302887201309204, 'eval_bleu': 5.4383, 'eval_gen_len': 16.0153, 'eval_runtime': 421.8028, 'eval_samples_per_second': 28.886, 'eval_steps_per_second': 3.611, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'train_runtime': 10903.968, 'train_samples_per_second': 37.991, 'train_steps_per_second': 4.749, 'train_loss': 0.18972355383442457, 'epoch': 3.0}


TrainOutput(global_step=51783, training_loss=0.18972355383442457, metrics={'train_runtime': 10903.968, 'train_samples_per_second': 37.991, 'train_steps_per_second': 4.749, 'total_flos': 3.28509444980736e+16, 'train_loss': 0.18972355383442457, 'epoch': 3.0})